In [3]:
# Initialize pyegeria

import os
view_server = os.environ.get("EGERIA_VIEW_SERVER","qs-view-server")
url = os.environ.get("EGERIA_VIEW_SERVER_URL","https://host.docker.internal:9443")
user_id = os.environ.get("EGERIA_USER", "peterprofile")
user_pwd = os.environ.get("EGERIA_USER_PASSWORD")
egeria_width = 150

from pyegeria import EgeriaTech, EgeriaCat, TEMPLATE_GUIDS, INTEGRATION_GUIDS
import asyncio
import nest_asyncio
nest_asyncio.apply()

from pyegeria import load_mermaid, render_mermaid, generate_process_graph
load_mermaid()

from commands.ops.monitor_integ_daemon_status import display_integration_daemon_status
from commands.ops.monitor_engine_activity_c import display_engine_activity_c
from commands.ops.monitor_gov_eng_status import display_gov_eng_status

In [ ]:
# print functions

def print_property(indent, property_name, property_value):
    print(indent + property_name + ": " + property_value)

def get_property(indent, property_name, properties):
    if properties:
       property_value=properties.get(property_name)
       if property_value:
           print_property(indent, property_name, property_value)

def print_element_header(indent, element_header):
    type_name = element_header["type"]["typeName"]
    guid = element_header["guid"]
    print(indent + type_name + " [" + guid + "]")

def print_properties(indent, property_name, properties):
    if properties:
        if type(properties) is str:
            print_property(indent, property_name, properties)
        elif type(properties) is int:
            print_property(indent, property_name, "{}".format(properties))
        elif type(properties) is dict:
            for key in properties.keys():
                print_properties(indent, key, properties[key])
        else:
            print(f"Funny property: type is {type(properties)}")
            
def print_element(indent, element):
    if element:
        print()
        print_element_header(indent + "", element.get("elementHeader"))
        print_properties(indent + " > ", "properties", element.get("properties"))
        print_properties(indent + " > ", "referenceableProperties", element.get("referenceableProperties"))

def print_related_elements(indent, related_elements):
    if related_elements:
        for related_element in related_elements:
            if related_element:
                print()
                print_element_header(indent, related_element.get("relationshipHeader"))
                print_properties(indent + " > ", "relationshipProperties", related_element.get("relationshipProperties"))
                print_element(indent + "   ", related_element.get("relatedElement"))
                                
def print_search_results(search_results):
    if search_results:
        if type(search_results) == str:
            print(search_results)
        else:
            for asset in search_results:
                if asset:
                    print()
                    print("Asset: ")
                    print_element_header(" > " , asset.get("elementHeader"))
                    print_properties(" > ", "properties", asset.get("properties"))
                    matchingElements = asset.get("matchingElements")
                    if matchingElements:
                        for matchingElement in matchingElements:
                            print("Matching Element:")
                            print_element_header("   > " , matchingElement.get("elementHeader"))
                            print_properties("   > " , "properties", matchingElement.get("properties"))

def print_external_id_map(catalog):
    if catalog:
        catalog_guid = catalog["elementHeader"]["guid"]
        print_element("", catalog)
        external_ids = egeria_tech.get_related_elements(catalog_guid, "ExternalIdScope")
        if external_ids:
            for external_id in external_ids:
                print()
                relationship_properties = external_id.get("relationshipProperties")
                if relationship_properties:
                    print_property(" > ", "Permitted Synchronization", relationship_properties.get("permittedSynchronization"))
                related_element = external_id.get("relatedElement")
                external_id_guid = related_element["elementHeader"]["guid"]
                open_metadata_elements = egeria_tech.get_related_elements(external_id_guid, "ExternalIdLink")
                print_related_elements("      ", open_metadata_elements)
                

In [ ]:

def print_governance_action_process_graph(process_graph):
    if process_graph:
        mermaid_graph = process_graph.get('mermaidGraph')
        if mermaid_graph:
            render_mermaid(mermaid_graph)
        process = process_graph.get('governanceActionProcess')
        if process:
            mermaid_specification = process.get('mermaidSpecification')
            if mermaid_specification:
                render_mermaid(mermaid_specification)



In [ ]:

def print_matching_gov_action_processes(egeria_tech, search_string):
    result=egeria_tech.find_gov_action_processes(search_string,starts_with=False, ends_with=False)
    if type(result) == str:
        print(result)
    else:
        for process in result:
            if process:
                process_guid=process['elementHeader']['guid']
                if process_guid:
                    print(process_guid)
                    process_graph = egeria_tech.get_gov_action_process_graph(process_guid)
                    print_governance_action_process_graph(process_graph)
                    

In [ ]:

    
def get_process_instances(egeria_tech, process_name):
    processGUID = egeria_tech.get_element_guid_by_unique_name(process_name)
    if processGUID == "No elements found":
        print(processGUID)
    else:
        processInstances = egeria_tech.get_related_elements(processGUID, "ProcessHierarchy")
        if processInstances:
            print("Process Instances:")
            guids = []
            for processInstance in processInstances:
                if processInstance:
                   relatedElement = processInstance.get('relatedElement')
                   if relatedElement:
                        guid = relatedElement["elementHeader"]["guid"]
                        guids.append(guid)
                        qualifiedName = relatedElement["properties"]["qualifiedName"]
                        print(f" * {qualifiedName} [{guid}]")
        return guids

def print_process_instances(egeria_client, process_name):
    process_instance_guids = get_process_instances(egeria_client, process_name)
    if process_instance_guids:
        for process_instance_guid in process_instance_guids:
            process_graph = egeria_client.get_gov_action_process_graph(process_instance_guid)
            print_governance_action_process_graph(process_graph)
            

In [4]:

# Asset Catalog OMVS

def print_asset_graph(egeria_tech, unique_name):
    guid=egeria_tech.get_element_guid_by_unique_name(unique_name)
    if guid == "No elements found":
        print(guid)
    else:
        print_asset_graph_by_guid(egeria_tech, guid)

def print_asset_graph_by_guid(egeria_tech, guid):
    graph_response = egeria_tech.get_asset_graph(guid)
    if graph_response:
        asset_graph = graph_response.get('mermaidGraph')
        isc_graph = graph_response.get('informationSupplyChainMermaidGraph')
        field_level_graph = graph_response.get('fieldLevelLineageGraph')
        if asset_graph:
            render_mermaid(asset_graph)
            if isc_graph:
                render_mermaid(isc_graph)
            if field_level_graph:
                render_mermaid(field_level_graph)
            


def print_asset_lineage_graph(egeria_tech, unique_name, limit_to_isc_qualified_name: str = None, hilight_isc_qualified_name: str = None):
    guid=egeria_tech.get_element_guid_by_unique_name(unique_name)
    if guid == "No elements found":
        print(guid)
    else:
        print_asset_lineage_graph_by_guid(egeria_tech, guid, limit_to_isc_qualified_name, hilight_isc_qualified_name)

def print_asset_lineage_graph_by_guid(egeria_tech, guid, limit_to_isc_qualified_name: str = None, hilight_isc_qualified_name: str = None):
    graph_response = egeria_tech.get_asset_lineage_graph(guid, limit_to_isc_q_name=limit_to_isc_qualified_name, hilight_isc_q_name=hilight_isc_qualified_name)
    if graph_response:
        asset_graph = graph_response.get('mermaidGraph')
        if asset_graph:
            render_mermaid(asset_graph)

def print_asset_lineage_edge_graph(egeria_tech, unique_name, limit_to_isc_qualified_name: str = None, hilight_isc_qualified_name: str = None):
    guid=egeria_tech.get_element_guid_by_unique_name(unique_name)
    if guid == "No elements found":
        print(guid)
    else:
        graph_response = egeria_tech.get_asset_lineage_graph(guid, limit_to_isc_q_name=limit_to_isc_qualified_name, hilight_isc_q_name=hilight_isc_qualified_name)
        if graph_response:
            asset_edge_graph = graph_response.get('edgeMermaidGraph')
            if asset_edge_graph:
                render_mermaid(asset_edge_graph)
                

In [ ]:

# Metadata Explorer OMVS

def print_anchor_graph(egeria_tech, unique_name):
    guid=egeria_tech.get_element_guid_by_unique_name(unique_name)
    if guid == "No elements found":
        print(guid)
    else:
        print_anchor_graph_by_guid(egeria_tech, guid)

def print_anchor_graph_by_guid(egeria_tech, guid):
        graph_response = egeria_tech.get_anchored_metadata_element_graph(guid)
        if graph_response:
            asset_graph = graph_response.get('mermaidGraph')
            if asset_graph:
                render_mermaid(asset_graph)

def print_all_related_metadata_elements(egeria_tech, guid):
    related_list = egeria_tech.get_all_related_metadata_elements(guid, {})
    if related_list:
        if type(related_list) == str:
            print(related_list)
        else:
            mermaid_graph = related_list.get('mermaidGraph')
            if mermaid_graph:
                render_mermaid(mermaid_graph)

def print_related_elements(egeria_tech, guid, relationship_type_name):
    related_list = egeria_tech.get_related_metadata_elements(guid, relationship_type_name, {})
    if type(related_list) == str:
        print(related_list)
    else:
        mermaid_graph = related_list.get('mermaidGraph')
        if mermaid_graph:
            render_mermaid(mermaid_graph)

def print_related_elements_for_name(egeria_tech, unique_name, relationship_type_name):
    guid=egeria_tech.get_element_guid_by_unique_name(unique_name)
    if guid == "No elements found":
        print(guid)
    else:
        print_related_elements(egeria_tech, guid, relationship_type_name)
                

In [ ]:

# Solution Architect OMVS


def print_information_supply_chains(egeria_tech, search_string, add_implementation=False, for_lineage=True):
    information_supply_chains=egeria_tech.find_information_supply_chains(search_string, add_implementation, body={"forLineage" : for_lineage})
    if information_supply_chains:
        if type(information_supply_chains) == str:
            print(information_supply_chains)
        else: 
            for information_supply_chain in information_supply_chains:
                if information_supply_chain:
                    isc_graph = information_supply_chain.get('mermaidGraph')
                    if isc_graph:
                        render_mermaid(isc_graph)

def print_solution_blueprints(egeria_tech, search_string):
    solution_blueprints=egeria_tech.find_solution_blueprints(search_string)
    if solution_blueprints:
        if type(solution_blueprints) == str:
            print(solution_blueprints)
        else:
            for solution_blueprint in solution_blueprints:
                if solution_blueprint:
                    blueprint_graph = solution_blueprint.get('mermaidGraph')
                    if blueprint_graph:
                        render_mermaid(blueprint_graph)


def print_solution_roles(egeria_tech, search_string):
    solution_roles=egeria_tech.find_solution_roles(search_string)
    if solution_roles:
        if type(solution_roles) == str:
            print(solution_roles)
        else:
            for solution_role in solution_roles:
                if solution_role:
                    role_graph = solution_role.get('mermaidGraph')
                    if role_graph:
                        render_mermaid(role_graph)


def print_solution_components(egeria_tech, search_string):
    solution_components=egeria_tech.find_solution_components(search_string)
    if solution_components:
        if type(solution_components) == str:
            print(solution_components)
        else:
            for solution_component in solution_components:
                if solution_component:
                    component_graph = solution_component.get('mermaidGraph')
                    if component_graph:
                        render_mermaid(component_graph)

In [ ]:

def print_project(egeria_tech, project_guid):
    element = egeria_tech.get_project_by_guid(project_guid)
    if element:
        if type(element) == str:
            print(element)
        else:
            mermaid_graph = element.get("mermaidGraph")
            render_mermaid(mermaid_graph)
            
def print_project_graph(egeria_tech, project_guid):
    graph = egeria_tech.get_project_graph(project_guid)
    if graph:
        if type(graph) == str:
            print(graph)
        else:
            mermaid_graph = graph.get("mermaidGraph")
            render_mermaid(mermaid_graph)
            

In [ ]:

def print_glossaries(egeria_tech, search_string):
    glossaries=egeria_tech.find_glossaries(search_string)
    if glossaries:
        if type(glossaries) == str:
            print(glossaries)
        else:
            for glossary in glossaries:
                if glossary:
                    mermaid_graph = glossary.get('mermaidGraph')
                    if mermaid_graph:
                        render_mermaid(mermaid_graph)


def print_glossary_categories(egeria_tech, search_string):
    categories=egeria_tech.find_glossary_categories(search_string)
    if categories:
        if type(categories) == str:
            print(categories)
        else:
            for category in categories:
                if category:
                    mermaid_graph = category.get('mermaidGraph')
                    if mermaid_graph:
                        render_mermaid(mermaid_graph)

def print_glossary_terms(egeria_tech, search_string):
    terms=egeria_tech.find_glossary_terms(search_string)
    if terms:
        if type(terms) == str:
            print(terms)
        else:
            for term in terms:
                if term:
                    mermaid_graph = term.get('mermaidGraph')
                    if mermaid_graph:
                        render_mermaid(mermaid_graph)
                        